In [20]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import cv2
import numpy as np
from sklearn.decomposition import PCA
import random
import pickle

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [31]:
def get_resnet_model_with_hooks():
    model = models.resnet50(pretrained=True)
    layers_to_hook = {
        'stage_1': 'maxpool',
        'stage_2': 'layer1',
        'stage_3': 'layer2',
        'stage_4': 'layer3',
        'stage_5': 'layer4',
        'final': 'fc'
    }

    features = {}
    def save_output(name):
        def hook(module, input, output):
            features[name] = output.detach()
        return hook

    for name, layer in layers_to_hook.items():
        getattr(model, layer).register_forward_hook(save_output(name))

    return model, features

model, features = get_resnet_model_with_hooks()

In [32]:
def process_video_pytorch(video_tensor, model, features):
    model.eval()
    with torch.no_grad():
        model(video_tensor)  # Forward pass, outputs are saved in features
    return {stage: feat.cpu().numpy() for stage, feat in features.items()}

In [21]:
def preprocess_video_pytorch(video_path, num_frames=30):
    vidcap = cv2.VideoCapture(video_path)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // num_frames)

    success, frame = vidcap.read()
    frames = []
    extracted_count = 0
    while success and extracted_count < num_frames:
        if total_frames > num_frames and (vidcap.get(cv2.CAP_PROP_POS_FRAMES) - 1) % frame_interval == 0:
            # Convert to PIL Image
            frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            # Resize and Normalize
            preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            frame = preprocess(frame)
            frames.append(frame)
            extracted_count += 1
        success, frame = vidcap.read()

    # Stack into a single tensor
    video_tensor = torch.stack(frames).to(device)
    return video_tensor

In [ ]:
def process_and_save_videos_pytorch(input_folder, output_folder, batch_size=90, id_range=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model, features = get_resnet_model_with_hooks()
    model = model.to(device)

    # ... rest of the function ...
    for filename in batch:
        # ... existing code to read and preprocess video ...
        stage_features = process_video_pytorch(video_tensor, model, features)

        for stage, feat in stage_features.items():
            stage_folder = os.path.join(output_folder, stage)
            if not os.path.exists(stage_folder):
                os.makedirs(stage_folder)
            output_file_path = os.path.join(stage_folder, f"{video_id}_{stage}.pkl")
            with open(output_file_path, 'wb') as file:
                pickle.dump(feat, file)


In [22]:
def get_resnet_model():
    model = models.resnet50(pretrained=True)
    features = list(model.children())[:-2]  # Remove the last two layers
    model = torch.nn.Sequential(*features)
    model = model.to(device)  # Transfer model to CUDA if available
    return model

In [23]:
def extract_features(video_tensor, model):
    model.eval()
    with torch.no_grad():
        features = model(video_tensor.to(device))  # Ensure tensor is on CUDA
    return features

In [24]:
def process_and_save_videos_pytorch(input_folder, output_folder, batch_size=90, id_range=None):
    # Ensure CUDA availability
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_resnet_model().to(device)

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get all MP4 files
    all_files = [f for f in os.listdir(input_folder) if f.endswith(".mp4")]

    # Filter files based on ID range if provided
    if id_range is not None:
        start_id, end_id = id_range
        all_files = [f for f in all_files if start_id <= f[:4] <= end_id]

    # Process in batches
    for i in range(0, len(all_files), batch_size):
        batch = all_files[i:i + batch_size]
        for filename in batch:
            video_id = filename[:4]
            video_path = os.path.join(input_folder, filename)

            # Check if the file for this video ID already exists in the output folder
            output_file_path = os.path.join(output_folder, f"{video_id}.pkl")
            if os.path.exists(output_file_path):
                print(f"Files for video ID {video_id} already exist. Skipping.")
                continue

            try:
                video_tensor = preprocess_video_pytorch(video_path)
                features = extract_features(video_tensor, model)

                with open(output_file_path, 'wb') as file:
                    pickle.dump(features.cpu().numpy(), file)
                    print(f"Saved file: {output_file_path}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

        print(f"Processed batch {i // batch_size + 1}/{(len(all_files) - 1) // batch_size + 1}")

In [25]:
def sample_and_apply_pca_pytorch(stage_folders, repo_root, variance=0.999):
    results = {}
    for stage in stage_folders:
        stage_folder = os.path.join(repo_root, stage)
        if not os.path.exists(stage_folder):
            print(f"Folder not found: {stage_folder}")
            continue

        files = os.listdir(stage_folder)
        if not files:
            print(f"No files found in {stage_folder}")
            continue

        # Randomly sample a few files (3 in this case)
        sampled_files = random.sample(files, min(len(files), 3))

        for file in sampled_files:
            file_path = os.path.join(stage_folder, file)
            with open(file_path, 'rb') as f:
                tensor = pickle.load(f)
                tensor = torch.Tensor(tensor)  # Convert to PyTorch tensor if necessary

                # Reshape tensor for PCA
                original_shape = tensor.shape
                tensor = tensor.reshape(tensor.shape[0], -1)

                # Apply PCA
                pca = PCA(n_components=variance)
                pca_tensor = pca.fit_transform(tensor)

                # Store results
                pca_shape = pca_tensor.shape
                results[file] = {
                    'original_shape': original_shape,
                    'pca_shape': pca_shape,
                    'explained_variance': variance
                }

    return results

In [26]:
input_folder = r"C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\Optical_flow\Video"
output_folder = r"C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\preprocessed_videos_30frames"
process_and_save_videos_pytorch(input_folder, output_folder, batch_size=50, id_range=('0001', '1000'))

C:\venv39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\venv39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Saved file: C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\preprocessed_videos_30frames\0001.pkl
Saved file: C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\preprocessed_videos_30frames\0002.pkl
Saved file: C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\preprocessed_videos_30frames\0003.pkl
Saved file: C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\preprocessed_videos_30frames\0004.pkl
Saved file: C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\preprocessed_videos_30frames\0005.pkl
Saved file: C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\preprocessed_videos_30frames\0006.pkl
Saved file: C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\preprocessed_videos_30frames\0007.pkl
Saved file: C:\Users\andre\OneDriv

In [29]:
# Define your stage folders and repo root
stage_folders = ['stage_1', 'stage_2', 'stage_3', 'stage_4', 'stage_5', 'final']
repo_root = r"C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow"

# Create the repo root folder if it doesn't exist
if not os.path.exists(repo_root):
    os.makedirs(repo_root)
    print(f"Created directory: {repo_root}")

# Create subfolders for each stage within the repo root
for folder in stage_folders:
    path = os.path.join(repo_root, folder)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Created directory: {path}")

In [30]:
pca_results = sample_and_apply_pca_pytorch(stage_folders, repo_root)

No files found in C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\stage_1
No files found in C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\stage_2
No files found in C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\stage_3
No files found in C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\stage_4
No files found in C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\stage_5
No files found in C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\resnet50-flow\final
